# Excelにsmileから描画した画像を表示

In [1]:
import pandas as pd
import pubchempy as pcp

properties = ['CanonicalSMILES', 'IUPACName']
benzene = pcp.get_properties(properties, 'benzene', 'name', as_dataframe=True)
toluene = pcp.get_properties(properties, 'toluene', 'name', as_dataframe=True)
df = pd.concat([benzene,toluene])
display(df)

,CanonicalSMILES,IUPACName
CID,,
241,C1=CC=CC=C1,benzene
1140,CC1=CC=CC=C1,toluene


In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
import openpyxl

def toImgFromSmiles(smiles, name):
    mol = Chem.MolFromSmiles(smiles)
    Draw.MolToFile(mol, f'{name}.png', size=(300, 300))

In [ ]:
from openpyxl.drawing.image import Image as XLImage

def add_image(worksheet, smiles_column, image_column):
    ws = worksheet
    # 行数を取得
    R_NUM = len(list(ws.rows))

    for i in range(1, R_NUM):
        idx = i + 1
        # smilesが入ってるカラム名(ex. A1, C5)
        smiles_col_name = smiles_column + str(idx)
        image_col_name = image_column + str(idx)

        smiles = ws[smiles_col_name].value
        if smiles == None: continue
        print(smiles)

        try:
            # 画像作成
            toImgFromSmiles(smiles, smiles_col_name)
            img = XLImage(f'{smiles_col_name}.png')
            img.width = 200
            img.height = 200

            # 画像を貼り付ける
            ws.add_image(img, image_col_name)

            # セルの高さを調整
            ws.row_dimensions[idx].height = img.height
        except FileNotFoundError as e:
            print('error ', e)
            # 画像ファイルが見つからない場合はスキップ
            pass

In [ ]:
import openpyxl

df.insert(2, 'img', '')
file_name = 'dataset.xlsx'

df.to_excel(file_name, index=False)
wb = openpyxl.load_workbook(file_name)
ws = wb.active

# Excelは左からA列,B列,C列,,,
add_image(ws, 'A', 'C')

# 保存
wb.save(file_name)